<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/acme.jpeg width=100 align="left">
<img src=images/vault-cluster1.png width=900 >

## Vaults Setup

**Shell alias setup**

In [ ]:
source ~/.zshrc

In [ ]:
alias |grep -i "evault[1-3]"

In [ ]:
alias |grep -i "evrd[1-3]"

### Launch 3 Vault servers, each on its own windows
  
Model is as follows
```
+---------------------------------+                    +------------------------------------+
| vault1 port:8200                |                    | vault2 port: 8202                  |
| Performance primary replication |    +----------->   | Performance secondary replication  |
| DR primary replication          |                    | (vault1 -> vault2)                 |
|                                 |                    |                                    |
+---------------------------------+                    +------------------------------------+
               +
               |
               v
+---------------------------------+
| vault3 port:8204                |
| DR secondary replication        |
| (vault1 -> vault3)              |
|                                 |
+---------------------------------+
```
Make sure you record the unseal key for each Vault server.


### License Vault (Optional)

In [ ]:
VAULT_ADDR="http://127.0.0.1:8200"; curl \
    --insecure \
    --header "X-Vault-Token: $VAULT_ROOT_TOKEN" \
    --request PUT \
    --data @/Users/tio/Documents/vault_license.json \
    $VAULT_ADDR/sys/license

### Vault1 - create users, policies and secrets on the primary cluster

In [ ]:
vault1 login root > /dev/null
echo "----"
vault1 auth enable userpass
echo '
path "*" {
    capabilities = ["create", "read", "update", "delete", "list", "sudo"]
}' | vault1 policy write vault-admin -

echo '
path "supersecret/*" {
  capabilities = ["list", "read"]
}' | vault1 policy write user -

vault1 write auth/userpass/users/vault password=vault policies=vault-admin
vault1 write auth/userpass/users/drtest password=drtest policies=user
echo "----"
vault1 secrets enable -path=supersecret generic

### Setup Performance Replication (Vault -> Vault2)

In [ ]:
vault1 login root > /dev/null
vault1 write -f sys/replication/performance/primary/enable
sleep 10
PRIMARY_PERF_TOKEN=$(vault1 write -format=json sys/replication/performance/primary/secondary-token id=vault2 \
  | jq --raw-output '.wrap_info .token' )
echo "---"
vault2 login root > /dev/null
vault2 write sys/replication/performance/secondary/enable token=${PRIMARY_PERF_TOKEN}

#### Validation 
Enable another secret engine on Vault1, configure it *not* to be replicated and perform validations:
* of Performance Replication on the Secondary cluser (Vault2)
* of Performance Replication on the Primary cluster (Vault1)


### Setup DR Replication (Vault1 -> Vault3)

In [ ]:
vault1 login root > /dev/null
vault1 write -f /sys/replication/dr/primary/enable
sleep 10
PRIMARY_DR_TOKEN=$(vault1 write -format=json /sys/replication/dr/primary/secondary-token id="vault3" | jq --raw-output '.wrap_info .token' )
vault3 login root > /dev/null
vault3 write /sys/replication/dr/secondary/enable token=${PRIMARY_DR_TOKEN}

#### Validation of DR

In [ ]:
vault3 read -format=json sys/replication/status |jq

### Promote DR Secondary (Vault3) to Primary Cluster
Shutdown Vault1

Generate operation token on Vault3

$ vault operator generate-root -dr-token -otp="vZUKRgIcHMgwNeB8T9jf0YwUvm" -decode="BXRjc2QLECsbfDQtJSEtVhxRHQRYPx8DGio"

s.686lYHS1SZkDonHhwbhfhVlG

curl -s http://127.0.0.1:8204/v1/sys/replication/dr/secondary/generate-operation-token/attempt | jq

Next generate OTP (one time pasword)
DR_OTP=$(vault3 operator generate-root -dr-token -generate-otp)

Create Nonce
NONCE=$(vault3 operator generate-root -dr-token -init -otp=${DR_OTP} |grep -i nonce | awk '{print $2}')



### Swing it back - Vault1 is back to become Primary

Run `vrd1` - starts empty Vault   
Configure Vault1 to become DR Secondary   
Demote Vault3  

### Promote DR Secondary (Vault2) to Primary

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 class="center">